# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [11]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [12]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,vardo,70.3705,31.1107,0.99,86,75,8.75,NO,1675177596
1,1,bluff,-46.6000,168.3333,15.43,97,100,2.59,NZ,1675177569
2,2,hobyo,5.3505,48.5268,25.26,71,2,9.23,SO,1675177569
3,3,lumsden,50.6501,-104.8678,-17.77,95,100,1.23,CA,1675177570
4,4,guerrero negro,27.9769,-114.0611,12.88,80,8,3.35,MX,1675177380


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [13]:
#%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = 'Humidity',
    color = 'City',
    alpha = .5

)

# Display the map
map_plot_1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [23]:
# Narrow down cities that fit criteria and drop any results with null values
clean_city_data = city_data_df.dropna()

clean_city_data = clean_city_data.loc[(clean_city_data["Max Temp"] > 20) & (clean_city_data["Max Temp"] < 40) & (clean_city_data["Wind Speed"] < 10) & (clean_city_data["Cloudiness"] == 0)]

# Display sample data
clean_city_data.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
15,15,bredasdorp,-34.5322,20.0403,24.75,62,0,7.07,ZA,1675177577
53,53,hermanus,-34.4187,19.2345,28.78,57,0,7.76,ZA,1675177621
71,71,acapulco,16.8634,-99.8901,24.90,78,0,3.09,MX,1675177420
80,80,hilo,19.7297,-155.0900,25.16,92,0,2.06,US,1675177637
94,94,cape town,-33.9258,18.4232,30.23,40,0,6.17,ZA,1675177363


### Step 3: Create a new DataFrame called `hotel_df`.

In [24]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = clean_city_data[['City','Country','Lat','Lng','Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name']= ""

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
15,bredasdorp,ZA,-34.5322,20.0403,62,
53,hermanus,ZA,-34.4187,19.2345,57,
71,acapulco,MX,16.8634,-99.8901,78,
80,hilo,US,19.7297,-155.0900,92,
94,cape town,ZA,-33.9258,18.4232,40,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [25]:
# Set parameters to search for a hotel
radius = 10000
params = {"categories": "accommodation.hotel",
    "apiKey": geoapify_key,
    "limit": 10
}

# Print a message to follow up the hotel search
print("Starting hotel search")


# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lon = row['Lng']
    lat = row['Lat']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = 'circle:' + str(lon) + ',' + str(lat) + ',' + str(radius)
    params["bias"] = 'proximity:' + str(lon) + ',' + str(lat)
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    
    # Make and API request using the params dictionaty
    name_address = requests.get(base_url,params=params)
    
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
bredasdorp - nearest hotel: Victoria Hotel
hermanus - nearest hotel: Aloe guest house
acapulco - nearest hotel: Hotel del Valle
hilo - nearest hotel: Dolphin Bay Hotel
cape town - nearest hotel: Townhouse Hotel
carnarvon - nearest hotel: No hotel found
arraial do cabo - nearest hotel: No hotel found
saint-pierre - nearest hotel: Tropic Hotel
geraldton - nearest hotel: Ocean Centre Hotel
comodoro rivadavia - nearest hotel: No hotel found
santa maria da vitoria - nearest hotel: Hotel Passarela
pochutla - nearest hotel: Posada San Jose
eyl - nearest hotel: فندق ايل دابي
moron - nearest hotel: La Esquina
presidencia roque saenz pena - nearest hotel: Hotel Presidente
mogadishu - nearest hotel: SYL Hotel
rajpur - nearest hotel: No hotel found
abu dhabi - nearest hotel: Marriott Downtown Abu-Dhabi
rio grande - nearest hotel: Hotel Vila Moura Executivo
chabahar - nearest hotel: هتل سپیده
champerico - nearest hotel: Hotel y Restaurante El Submarino
puerto escondido - neare

,City,Country,Lat,Lng,Humidity,Hotel Name
15,bredasdorp,ZA,-34.5322,20.0403,62,Victoria Hotel
53,hermanus,ZA,-34.4187,19.2345,57,Aloe guest house
71,acapulco,MX,16.8634,-99.8901,78,Hotel del Valle
80,hilo,US,19.7297,-155.0900,92,Dolphin Bay Hotel
94,cape town,ZA,-33.9258,18.4232,40,Townhouse Hotel
110,carnarvon,AU,-24.8667,113.6333,81,No hotel found
123,arraial do cabo,BR,-22.9661,-42.0278,58,No hotel found
139,saint-pierre,RE,-21.3393,55.4781,74,Tropic Hotel
169,geraldton,AU,-28.7667,114.6000,78,Ocean Centre Hotel
176,comodoro rivadavia,AR,-45.8667,-67.5000,36,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [26]:
#%%capture --no-display

#Cleaning up hotel_df to remove rows with no hotel found
clean_hotel_df = hotel_df[hotel_df['Hotel Name'] != 'No hotel found']
clean_hotel_df.head()

# Configure the map plot
map_plot_1 = clean_hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = 'Humidity',
    color = 'City',
    alpha = .5,
    hover_cols = ['Country','Hotel Name']   
)

# Display the map
map_plot_1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name)